In [1]:
%cd ../

/u02/thanhhm/research-idea


In [2]:
import plasma.torch as ptorch
import torch.optim as opts
import numpy as np
import plasma.torch.training as training
import plasma.logging as plog

from torch import Tensor
from torch.utils.data import DataLoader
from plasma.torch.training.prototypes.trainer import TrainerState

In [3]:
@training.wrappers.NanChecker()
@training.wrappers.Tensorboard('test')
@training.wrappers.GradientClipping()
@training.wrappers.AMP()
class TestTrainer(ptorch.training.BaseTrainer):

    def __init__(self):
        super().__init__()

        self.max_epoch = 2

    def init_state(self) -> TrainerState:
        model = ptorch.nn.Linear(1, 1)

        return TrainerState(
            epoch=-1,
            model=model,
            optimizer=opts.SGD(model.parameters()),
            scheduler=None,
        )
    
    def init_train_loader(self) -> DataLoader:
        ds = ptorch.data.AdhocData([1.0]*10, lambda x: x)
        return ds.get_torch_loader(batch_size=1, workers=0)
    
    def forward(self, state: TrainerState, i: int, inputs):
        model = state.model
        model.zero_grad()

        y = model(inputs[np.newaxis].float())

        return (y - inputs).mean()
    
    def backward(self, state: TrainerState, bjective_val: Tensor):
        bjective_val.backward()
        state.optimizer.step()

In [4]:
trainer = TestTrainer()

In [ ]:
trainer.run()